Task 1 find the primes numbers

Given a positive integer and an list of prime numbers, look for the two prime numbers, that sum the positive number. Consider an appropriate number of qubits and explain why your proposal is valid for all kinds of numbers in case 

def find_the_primes_numbers (int:number_1, list[int] ,number_2):
     “””
    number_1 : integer value that is the positive number to decompose,
    number_2 : integer list that has two prime numbers to add to obtain number_1.
    Return the number_a and number_b
     “””

Example:

A = find_the_primes_numbers (18,[1,3,5,7,11,13,15])
print(A)

“5,13”


Explanation: 
In this solution, I assume the summands to be 4-bit binary numbers and the sum to be a 5-bit binary number.
The circuit conists of 8 qubits (4 qubits per number) and 9 classical bits(5 for sum, 4 for carry).
I have provided the comments in the solution for better understanding.

In [43]:
#importing required libraries
from qiskit import *
from qiskit.visualization import plot_histogram
%matplotlib inline
from qiskit.circuit.library.arithmetic import WeightedAdder

In [44]:
# Defining the main function
def find_the_primes_numbers(number_1, primelist):
   # Shortening the list, since the possible pairs of summands will only consist of prime numbers less than the target number 
   for i in range(len(primelist)):
      if (primelist[i]>number_1):
         primelist=primelist[:i]
         break
   # Checking for every pair, whether the sum equals target or not
   for i in range(len(primelist)):
      for j in range(i+1,len(primelist)):
         if add_numbers(tobinary(primelist[i]),tobinary(primelist[j]))==number_1:
            return (primelist[i],primelist[j])
   # If no such pair is found
   return "No pair of prime numbers from given list sum up to the target number"


def tobinary(num):
   # Converting decimal number to binary string
   s=bin(num).replace("0b","")
   # Ensure the string is 4 digits long
   while (len(s)!=4):
      s='0'+s
   return s

         

In [45]:
def add_numbers(num1,num2)->int:
    # Create a weighted adder circuit
    circuit=WeightedAdder(8,[1,2,4,8,1,2,4,8])

    # Create classical registers for sum and carry
    sum_res=ClassicalRegister(5, name='sum_res')
    carry_res=ClassicalRegister(4,name='carry_res')

    # Add registers to the circuit
    circuit.add_register(sum_res)
    circuit.add_register(carry_res)

    # Measure the sum and carry registers
    q_sum=circuit.qregs[1]
    q_carry=circuit.qregs[2]
    circuit.measure(q_sum,sum_res)
    circuit.measure(q_carry,carry_res)

    # Create quantum circuits for input numbers
    val1=QuantumCircuit(4,name='val1')
    val2=QuantumCircuit(4, name='val2')

    # Applying X gates to represent the input numbers as quantum states
    for i in range(len(num1)):
        if num1[i]=='1':
            val1.x(len(num1)-i-1)
        if num2[i]=='1':
            val2.x(len(num2)-i-1)

    # Compose input number circuits with the weighted adder circuit by adding them in front of the circuit
    circuit.compose(val1.to_gate(), qubits=[0,1,2,3], front=True, inplace=True)
    circuit.compose(val2.to_gate(), qubits=[4,5,6,7], front=True, inplace=True)

    # Execute the circuit on a quantum simulator
    backend=Aer.get_backend('qasm_simulator')
    result=execute(circuit,backend,shots=10).result()

    # Get the output result
    output=list(result.get_counts().keys())[0]

    # Reversing the number to calculate the resultant decimal number
    number=output[-1:-6:-1]
    n=0
    for i in range(len(number)):
        if number[i]=='1':
            n+=2**i
    # Returning the sum of the numbers
    return n

In [47]:
A=find_the_primes_numbers(18,[2,3,5,7,11,13])
print(A)

(5, 13)


In [48]:
B=find_the_primes_numbers(10,[2,3,5,7,11,13])
print(B)

(3, 7)


In [49]:
C=find_the_primes_numbers(6,[2,3,5,7,11,13])
print(C)

No pair of prime numbers from given list sum up to the target number
